In [1]:
%env PGDATABASE=postgres
%env PGHOST={ip address}
%env PGPORT=5432
%env PGUSER=postgres
%env PGPASSWORD={password}
%env PGINSTANCE={pg instance}

env: PGDATABASE=postgres
env: PGHOST=10.106.209.5
env: PGPORT=5432
env: PGUSER=postgres
env: PGPASSWORD=cs327e
env: PGINSTANCE=postgres


### Populate Database

In [2]:
!psql -c "\i pg_college_staging.sql"

psql:pg_college_staging.sql:1: NOTICE:  drop cascades to 3 other objects
DETAIL:  drop cascades to table college.classes
drop cascades to table college.current_students
drop cascades to table college.new_students
DROP SCHEMA
CREATE SCHEMA
SET
CREATE TABLE
CREATE TABLE
CREATE TABLE


In [3]:
!gcloud sql import csv $PGINSTANCE gs://cs327e-open-access/college_staging/Classes.csv --database=postgres --table=college.Classes --quiet

Importing data into Cloud SQL instance...done.                                 
Imported data from [gs://cs327e-open-access/college_staging/Classes.csv] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/cs327e-sp2020/instances/postgres].


In [4]:
!gcloud sql import csv $PGINSTANCE gs://cs327e-open-access/college_staging/Current_Students.csv --database=postgres --table=college.Current_Students --quiet

Importing data into Cloud SQL instance...done.                                 
Imported data from [gs://cs327e-open-access/college_staging/Current_Students.csv] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/cs327e-sp2020/instances/postgres].


In [5]:
!gcloud sql import csv $PGINSTANCE gs://cs327e-open-access/college_staging/New_Students.csv --database=postgres --table=college.New_Students --quiet

Importing data into Cloud SQL instance...done.                                 
Imported data from [gs://cs327e-open-access/college_staging/New_Students.csv] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/cs327e-sp2020/instances/postgres].


### Verify data load

In [6]:
!psql -c "select count(*) from college.Classes"

 count 
-------
    12
(1 row)



In [7]:
!psql -c "select count(*) from college.Current_Students"

 count 
-------
    13
(1 row)



In [8]:
!psql -c "select count(*) from college.New_Students"

 count 
-------
     7
(1 row)



In [9]:
!psql -c "select * from college.Classes limit 5"

   tid   |  instructor   |       dept       |  cno   |              cname               | credits 
---------+---------------+------------------+--------+----------------------------------+---------
 fares   | Fares Fraij   | CS               | CS331E | Elements of Software Engineering |       3
 mitra   | Shyamal Mitra | CS               | CS313E | Elements of Software Design      |       3
 mitra   | Shyamal Mitra | Computer Science | CS329E | Elements of Web Programming      |       3
 cannata | Phil Cannata  | CS               | CS326E | Elements of Networking           |       3
 scohen  | Shirley Cohen | Computer Science | CS327E | Elements of Databases            |       3
(5 rows)



In [10]:
!psql -c "select * from college.Current_Students limit 5"

  sid   | fname | lname |    dob     |  cno   |            cname            | credits | grade 
--------+-------+-------+------------+--------+-----------------------------+---------+-------
 kev18  | Kevin | Lin   | 1999-05-10 | CS329E | Elements of Web Programming |       3 | 
 paulg  | Paul  | Gore  | 2000-09-17 | CS329E | Elements of Web Programming |       3 | A 
 paulg  | Paul  | Gore  | 2000-09-17 | CS326E | Elements of Networking      |       3 | A-
 paulg  | Paul  | Gore  | 2000-09-17 | CS313E | Elements of Software Design |       3 | 
 bzen26 | Biswa | Zen   | 04/22/1998 | CS313E | Elements of Software Design |       3 | B+
(5 rows)



In [11]:
!psql -c "select * from college.New_Students limit 5"

   sid    |  fname  |  lname   |    dob     
----------+---------+----------+------------
 aprilz   | April   | Lopez    | 2000-01-10
 sudeepa4 | Sudeepa | Roy      | 2000-10-01
 ryan22   | Ryan    | Watson   | 2000-11-07
 dchen    | David   | Chen     | 2000-08-22
 natea    | Nathan  | Anderson | 2000-04-08
(5 rows)



### Remodel tables

In [13]:
!psql -c "update college.Current_Students set dob = '1999-01-03' where sid = 'jerryh'"

UPDATE 3


In [15]:
!psql -c "update college.Current_Students set dob = '2000-01-10' where sid = 'aprilz'"

UPDATE 1


In [17]:
!psql -c "update college.Current_Students set dob = '1998-04-22' where sid = 'bzen26'"

UPDATE 1


In [18]:
!psql -c "select * from college.Current_Students"

   sid    |  fname  |  lname   |    dob     |  cno   |                   cname                   | credits | grade 
----------+---------+----------+------------+--------+-------------------------------------------+---------+-------
 kev18    | Kevin   | Lin      | 1999-05-10 | CS329E | Elements of Web Programming               |       3 | 
 paulg    | Paul    | Gore     | 2000-09-17 | CS329E | Elements of Web Programming               |       3 | A 
 paulg    | Paul    | Gore     | 2000-09-17 | CS326E | Elements of Networking                    |       3 | A-
 paulg    | Paul    | Gore     | 2000-09-17 | CS313E | Elements of Software Design               |       3 | 
 jc       | James   | Cowe     | 2000-04-22 | CS327E | CS327E – Elements of Databases            |       3 | 
 jc       | James   | Cowe     | 2000-04-22 | CS331E | CS331E - Elements of Software Engineering |       3 | 
 jc       | James   | Cowe     | 2000-04-22 | CS313E | CS313E - Elements of Software Design      |      

In [19]:
!psql -c "create table college.Student as \
select distinct sid, fname, lname, dob, 'PRO' as status \
from college.New_Students \
union \
select distinct sid, fname, lname, cast(dob as date), 'CUR' as status \
from college.Current_Students"

SELECT 14


In [20]:
!psql -c "select * from college.Student order by sid"

   sid    |  fname  |  lname   |    dob     | status 
----------+---------+----------+------------+--------
 aprilz   | April   | Lopez    | 2000-01-10 | CUR
 aprilz   | April   | Lopez    | 2000-01-10 | PRO
 bzen26   | Biswa   | Zen      | 1998-04-22 | CUR
 dchen    | David   | Chen     | 2000-08-22 | PRO
 jc       | James   | Cowe     | 2000-04-22 | CUR
 jerryh   | Jerry   | Hargrove | 1999-01-03 | CUR
 kev18    | Kevin   | Lin      | 1999-05-10 | CUR
 kgomez9  | Katie   | Gomez    | 2001-03-03 | PRO
 natea    | Nathan  | Anderson | 2000-04-08 | PRO
 nitinm   | Nitin   | Mogal    | 1999-07-31 | PRO
 paulg    | Paul    | Gore     | 2000-09-17 | CUR
 ryan22   | Ryan    | Watson   | 2000-11-07 | PRO
 sudeepa4 | Sudeepa | Roy      | 2000-10-01 | CUR
 sudeepa4 | Sudeepa | Roy      | 2000-10-01 | PRO
(14 rows)



In [21]:
!psql -c "delete from college.Student where sid = 'aprilz' and status = 'PRO'"

DELETE 1


In [22]:
!psql -c "delete from college.Student where sid = 'sudeepa4' and status = 'PRO'"

DELETE 1


In [24]:
!psql -c "alter table college.Student add primary key(sid)"

ALTER TABLE


In [27]:
!psql -c "select * from college.Classes order by cno"

   tid   |  instructor   |       dept       |  cno   |              cname               | credits 
---------+---------------+------------------+--------+----------------------------------+---------
 mitra   | Shyamal Mitra | CS               | CS313E | Elements of Software Design      |       3
 bulko   | Bill Bulko    | Computer Science | CS313E | Elements of Software Design      |       3
 cannata | Phil Cannata  | CS               | CS326E | Elements of Networking           |       3
 scohen  | Shirley Cohen | Computer Science | CS327E | Elements of Databases            |       3
 mitra   | Shyamal Mitra | Computer Science | CS329E | Elements of Web Programming      |       3
 fares   | Fares Fraij   | CS               | CS331E | Elements of Software Engineering |       3
 cannata | Phil Cannata  | CS               | CS347  | Data Management                  |       3
 downing | Glenn Downing | CS               | CS373  | Software Engineering             |       3
 koch    | Hans Ko

In [28]:
!psql -c "update college.Classes set dept = 'Computer Science' where dept = 'CS'"

UPDATE 5


In [30]:
!psql -c "update college.Classes set dept = 'Mathematics' where dept = 'Math'"

UPDATE 2


In [31]:
!psql -c "create table college.Teacher as select distinct tid, \
instructor, dept from college.Classes"

SELECT 10


In [34]:
!psql -c "select * from college.Teacher order by tid"

   tid   |  instructor   |       dept       
---------+---------------+------------------
 bulko   | Bill Bulko    | Computer Science
 cannata | Phil Cannata  | Computer Science
 downing | Glenn Downing | Computer Science
 fares   | Fares Fraij   | Computer Science
 koch    | Hans Koch     | Mathematics
 mitra   | Shyamal Mitra | Computer Science
 mueller | Peter Mueller | Mathematics
 neeman  | Joe Neeman    | Mathematics
 scohen  | Shirley Cohen | Computer Science
 tran    | Ngoc Tran     | Mathematics
(10 rows)



In [36]:
!psql -c "alter table college.Teacher add primary key (tid)"

ALTER TABLE


In [37]:
!psql -c "create table college.Class as \
select distinct cno, cname, credits \
from college.Classes"

SELECT 10


In [38]:
!psql -c "select * from college.Class order by cno"

  cno   |              cname               | credits 
--------+----------------------------------+---------
 CS313E | Elements of Software Design      |       3
 CS326E | Elements of Networking           |       3
 CS327E | Elements of Databases            |       3
 CS329E | Elements of Web Programming      |       3
 CS331E | Elements of Software Engineering |       3
 CS347  | Data Management                  |       3
 CS373  | Software Engineering             |       3
 M328K  | Intro to Number Theory           |       3
 M358K  | Applied Statistics               |       3
 M362K  | Probability I                    |       3
(10 rows)



In [40]:
!psql -c "alter table college.Class add primary key (cno)"

ALTER TABLE


In [41]:
!psql -c "create table college.Teaches as \
select distinct tid, cno \
from college.Classes"

SELECT 12


In [42]:
!psql -c "alter table college.Teaches add primary key(tid, cno)"

ALTER TABLE


In [43]:
!psql -c "create table college.Takes as \
select distinct sid, cno, grade \
from college.Current_Students \
where cno != ''"

SELECT 11


In [45]:
!psql -c "select * from college.Takes order by sid, cno"

  sid   |  cno   | grade 
--------+--------+-------
 bzen26 | CS313E | B+
 jc     | CS313E | 
 jc     | CS327E | 
 jc     | CS331E | 
 jerryh | CS327E | B 
 jerryh | CS327E | 
 jerryh | CS329E | A-
 kev18  | CS329E | 
 paulg  | CS313E | 
 paulg  | CS326E | A-
 paulg  | CS329E | A 
(11 rows)



In [46]:
!psql -c "delete from college.Takes where sid = 'jerryh' and grade is null"

DELETE 1


In [47]:
!psql -c "alter table college.Takes add primary key (sid, cno)"

ALTER TABLE


In [48]:
!psql -c "alter table college.Takes add foreign key (sid) references college.Student (sid)"

ALTER TABLE


In [49]:
!psql -c "alter table college.Takes add foreign key (cno) references college.Class (cno)"

ALTER TABLE


In [50]:
!psql -c "alter table college.Teaches add foreign key (tid) references college.Teacher (tid)"

ALTER TABLE


In [52]:
!psql -c "alter table college.Teaches add foreign key (cno) references college.Class (cno)"

ALTER TABLE


In [54]:
!psql -c "\d college.Student"

           Table "college.student"
 Column |          Type          | Modifiers 
--------+------------------------+-----------
 sid    | character varying(50)  | not null
 fname  | character varying(200) | 
 lname  | character varying(200) | 
 dob    | date                   | 
 status | text                   | 
Indexes:
    "student_pkey" PRIMARY KEY, btree (sid)
Referenced by:
    TABLE "college.takes" CONSTRAINT "takes_sid_fkey" FOREIGN KEY (sid) REFERENCES college.student(sid)



In [55]:
!psql -c "\d college.Class"

            Table "college.class"
 Column  |          Type          | Modifiers 
---------+------------------------+-----------
 cno     | character(6)           | not null
 cname   | character varying(200) | 
 credits | integer                | 
Indexes:
    "class_pkey" PRIMARY KEY, btree (cno)
Referenced by:
    TABLE "college.takes" CONSTRAINT "takes_cno_fkey" FOREIGN KEY (cno) REFERENCES college.class(cno)
    TABLE "college.teaches" CONSTRAINT "teaches_cno_fkey" FOREIGN KEY (cno) REFERENCES college.class(cno)



In [56]:
!psql -c "\d college.Teacher"

             Table "college.teacher"
   Column   |          Type          | Modifiers 
------------+------------------------+-----------
 tid        | character varying(50)  | not null
 instructor | character varying(200) | 
 dept       | character varying(50)  | 
Indexes:
    "teacher_pkey" PRIMARY KEY, btree (tid)
Referenced by:
    TABLE "college.teaches" CONSTRAINT "teaches_tid_fkey" FOREIGN KEY (tid) REFERENCES college.teacher(tid)



In [57]:
!psql -c "\d college.Takes"

           Table "college.takes"
 Column |         Type          | Modifiers 
--------+-----------------------+-----------
 sid    | character varying(50) | not null
 cno    | character(6)          | not null
 grade  | character(2)          | 
Indexes:
    "takes_pkey" PRIMARY KEY, btree (sid, cno)
Foreign-key constraints:
    "takes_cno_fkey" FOREIGN KEY (cno) REFERENCES college.class(cno)
    "takes_sid_fkey" FOREIGN KEY (sid) REFERENCES college.student(sid)



In [58]:
!psql -c "\d college.Teaches"

          Table "college.teaches"
 Column |         Type          | Modifiers 
--------+-----------------------+-----------
 tid    | character varying(50) | not null
 cno    | character(6)          | not null
Indexes:
    "teaches_pkey" PRIMARY KEY, btree (tid, cno)
Foreign-key constraints:
    "teaches_cno_fkey" FOREIGN KEY (cno) REFERENCES college.class(cno)
    "teaches_tid_fkey" FOREIGN KEY (tid) REFERENCES college.teacher(tid)

